Create Player Ratings

In [1]:
import pandas as pd
import re
import numpy as np

import os
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

In [14]:
player_stats = pd.read_csv("/Users/ciaran/Documents/Projects/AFL/data/scored_player_stats.csv")
player_stats['Season'] = player_stats['Match_ID'].apply(lambda x: int(x[:4]))
player_stats['Player_Season'] = player_stats['Player'] + "_" + player_stats['Season'].astype(str)

player_stats['Score'] = player_stats['Behinds'] + player_stats['Goals']*6
player_stats['xScore_Diff'] = player_stats['Score'] - player_stats['xScore']

player_stats['Home_Team'] = player_stats['Match_ID'].apply(lambda x: x.split("_")[1]).str.replace(r"([A-Z])", r" \1").str.strip()
player_stats['Away_Team'] = player_stats['Match_ID'].apply(lambda x: x.split("_")[-1]).str.replace(r"([A-Z])", r" \1").str.strip()
player_stats['Opponent'] = np.where(player_stats['Team'] == player_stats['Home_Team'], player_stats['Away_Team'], player_stats['Home_Team'])

finals_round_map = {'F1':24,
                    'F2':25,
                    'F3':26,
                    'F4':27}
player_stats['Round_ID'] = player_stats['Round_ID'].astype(str)
player_stats['Round'] = (player_stats['Round_ID'].str[4:].replace(finals_round_map)).astype(int)
player_stats['Round_str'] = player_stats['Round_ID'].str[4:].replace(finals_round_map)
player_stats['Round_ID_num'] = (player_stats['Season'].astype(str) + player_stats['Round'].astype(str)).astype(int)

player_stats.tail()

,Match_ID,Team,Player,Round_ID,AFL_API_Player_ID,Player_Type,playerId,Age,Height,Weight,Number,Kicking_Foot,State_Of_Origin,Draft_Year,Debut_Year,Recruited_From,Draft_Position,Draft_Type,Photo_URL,Date_Of_Birth,Percent_Played,Behinds,Bounces,Centre_Bounces_Attended,Centre_Clearances,Clangers,Defensive_Contest_Losses,Defensive_Contest_Loss_Percentage,Defensive_One_On_One_Contests,Contested_Marks,Contested_Possession_Rate,Contested_Possessions,Offensive_One_On_One_Contests,Offensive_Contest_Wins,Offensive_Contest_Win_Percentage,Defensive_Half_Pressure_Acts,Disposal_Efficiency,Disposals,AFL_Fantasy_Points,Effective_Disposals,Effective_Kicks,Inside_50_Ground_Ball_Gets,Frees_Against,Frees_For,Goal_Accuracy,Goal_Assists,Goals,Ground_Ball_Gets,Handballs,Hit_Outs,Hit_Outs_To_Advantage,Hit_Outs_To_Advantage_Rate,Hit_Out_Win_Percentage,Inside_50s,Intercept_Marks,Intercepts,Kick_Efficiency,Kick_Ins,Kick_Ins_Played_On,Kicks,Kick_To_Handball_Ratio,Marks,Marks_Inside_50,Marks_On_Lead,Metres_Gained,One_Percenters,Pressure_Acts,Player_Rating_Points,Rebound_50s,Ruck_Contests,Score_Involvements,Score_Launches,Shots_At_Goal,Spoils,Stoppage_Clearances,Tackles,Tackles_Inside_50,Clearances,Possessions,Turnovers,Uncontested_Possessions,AFLCA_Player_ID,Coaches_Votes,Position,Team_Status,Position_Sub_Group,Position_Group,Year,Brownlow_Votes,Season,xScore,xT_created,xT_denied,vaep_value,offensive_value,defensive_value,exp_vaep_value,exp_offensive_value,exp_defensive_value,xT_received,xT_prevented,vaep_value_received,exp_vaep_value_received,Player_Season,Score,xScore_Diff,Home_Team,Away_Team,Opponent,Round,Round_str,Round_ID_num
27846,202323_WesternBulldogs_WestCoast,Western Bulldogs,Oskar Baker,202323,Oskar_Baker,MIDFIELDER,CD_I1008855,25,184,87,13,RIGHT,QLD,2017.0,2019.0,Wilston Grange (Qld)/Aspley (NEAFL)/Melbourne,48.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1998-05-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Half Forward Flank Right,FINAL_TEAM,Half-Forward,Forward,NaN,NaN,2023,7.959769,-0.322645,0.016709,0.399113,0.700571,-0.301458,2.852330,3.819445,-0.967115,-0.326746,0.071312,1.048314,3.462521,Oskar Baker_2023,NaN,NaN,Western Bulldogs,West Coast,West Coast,23,23,202323
27847,202323_WesternBulldogs_WestCoast,Western Bulldogs,Rhylee West,202323,Rhylee_West,MEDIUM_FORWARD,CD_I1006127,23,183,82,14,RIGHT,VIC,2018.0,2019.0,Strathmore (Vic)/St Kevin&apos;s College (Vic)...,26.0,fatherSon,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,2000-07-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Forward Pocket Right,FINAL_TEAM,Forward-Pocket,Forward,NaN,NaN,2023,9.770643,0.138877,0.000000,0.569370,0.645190,-0.075820,7.074921,7.476112,-0.401191,0.486820,0.004338,0.926924,6.220417,Rhylee West_2023,NaN,NaN,Western Bulldogs,West Coast,West Coast,23,23,202323
27848,202323_WesternBulldogs_WestCoast,Western Bulldogs,Rory Lobb,202323,Rory_Lobb,KEY_FORWARD,CD_I990740,30,207,106,7,RIGHT,WA,2013.0,2014.0,Bassendean (WA)/Swan Districts (WAFL)/GWS/Frem...,29.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1993-02-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Forward Pocket Left,FINAL_TEAM,Forward-Pocket,Forward,NaN,NaN,2023,17.116590,-0.143941,-0.002520,1.729477,1.851836,-0.122359,11.169317,12.752323,-1.583006,0.487798,-0.000221,1.150055,10.551092,Rory Lobb_2023,NaN,NaN,Western Bulldogs,West Coast,West Coast,23,23

In [3]:
player_stats[['Position_Group', 'Position_Sub_Group', 'Position']].value_counts().sort_index()

Position_Group  Position_Sub_Group  Position                
Back            Back-Pocket         Back Pocket Left            1178
                                    Back Pocket Right           1167
                Half-Back           Half Back Flank Left        1165
                                    Half Back Flank Right       1157
                Key-Back            Centre Half Back            1159
                                    Full Back                   1181
Forward         Forward-Pocket      Forward Pocket Left         1160
                                    Forward Pocket Right        1169
                Half-Forward        Half Forward Flank Left     1154
                                    Half Forward Flank Right    1145
                Key-Forward         Centre Half Forward         1152
                                    Full Forward                1191
Interchange     Interchange         Interchange                 4651
                                    Substi

Different Positions are expected to do different things

In [15]:
match = player_stats[player_stats['Match_ID'] == "202323_WesternBulldogs_WestCoast"]
match.head()

,Match_ID,Team,Player,Round_ID,AFL_API_Player_ID,Player_Type,playerId,Age,Height,Weight,Number,Kicking_Foot,State_Of_Origin,Draft_Year,Debut_Year,Recruited_From,Draft_Position,Draft_Type,Photo_URL,Date_Of_Birth,Percent_Played,Behinds,Bounces,Centre_Bounces_Attended,Centre_Clearances,Clangers,Defensive_Contest_Losses,Defensive_Contest_Loss_Percentage,Defensive_One_On_One_Contests,Contested_Marks,Contested_Possession_Rate,Contested_Possessions,Offensive_One_On_One_Contests,Offensive_Contest_Wins,Offensive_Contest_Win_Percentage,Defensive_Half_Pressure_Acts,Disposal_Efficiency,Disposals,AFL_Fantasy_Points,Effective_Disposals,Effective_Kicks,Inside_50_Ground_Ball_Gets,Frees_Against,Frees_For,Goal_Accuracy,Goal_Assists,Goals,Ground_Ball_Gets,Handballs,Hit_Outs,Hit_Outs_To_Advantage,Hit_Outs_To_Advantage_Rate,Hit_Out_Win_Percentage,Inside_50s,Intercept_Marks,Intercepts,Kick_Efficiency,Kick_Ins,Kick_Ins_Played_On,Kicks,Kick_To_Handball_Ratio,Marks,Marks_Inside_50,Marks_On_Lead,Metres_Gained,One_Percenters,Pressure_Acts,Player_Rating_Points,Rebound_50s,Ruck_Contests,Score_Involvements,Score_Launches,Shots_At_Goal,Spoils,Stoppage_Clearances,Tackles,Tackles_Inside_50,Clearances,Possessions,Turnovers,Uncontested_Possessions,AFLCA_Player_ID,Coaches_Votes,Position,Team_Status,Position_Sub_Group,Position_Group,Year,Brownlow_Votes,Season,xScore,xT_created,xT_denied,vaep_value,offensive_value,defensive_value,exp_vaep_value,exp_offensive_value,exp_defensive_value,xT_received,xT_prevented,vaep_value_received,exp_vaep_value_received,Player_Season,Score,xScore_Diff,Home_Team,Away_Team,Opponent,Round,Round_str,Round_ID_num
27805,202323_WesternBulldogs_WestCoast,West Coast,Alex Witherden,202323,Alex_Witherden,MEDIUM_DEFENDER,CD_I998128,24,188,85,23,RIGHT,VIC,2016.0,2017.0,"St Joseph&apos;s, Geel (Vic)/Geel College (Vic...",23.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1998-09-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Back Pocket Right,FINAL_TEAM,Back-Pocket,Back,NaN,NaN,2023,0.000000,-0.267964,0.023892,1.114284,0.536018,0.578265,8.365421,5.327914,3.037508,-1.010768,0.023430,1.118252,5.976717,Alex Witherden_2023,NaN,NaN,Western Bulldogs,West Coast,Western Bulldogs,23,23,202323
27806,202323_WesternBulldogs_WestCoast,West Coast,Andrew Gaff,202323,Andrew_Gaff,MIDFIELDER,CD_I290801,31,184,83,3,LEFT,VIC,2010.0,2011.0,Kew (Vic)/Carey Grammar (Vic)/Oakleigh U18,4.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1992-06-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Wing Left,FINAL_TEAM,Wing,Midfield,NaN,NaN,2023,2.148563,0.191014,0.052414,0.631304,0.340834,0.290469,4.194502,3.959685,0.234816,-0.230022,0.138102,0.953913,7.855344,Andrew Gaff_2023,NaN,NaN,Western Bulldogs,West Coast,Western Bulldogs,23,23,202323
27807,202323_WesternBulldogs_WestCoast,West Coast,Bailey J. Williams,202323,Bailey_J._Williams,RUCK,CD_I1006114,23,201,101,32,RIGHT,VIC,2018.0,2020.0,Pines (Vic)/Western Port SC (Vic)/Dandenong U18,35.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,2000-04-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ruck,FINAL_TEAM,Ruck,Ruck,NaN,NaN,2023,0.000000,0.251066,0.067486,0.449135,0.451447,-0.002312,4.723950,4.275080,0.448870,-0.014674,0.022624,-0.076412,3.294606,Bailey J. Williams_2023,NaN,NaN,Western Bulldogs,West Coast,Western Bulldogs,23,23,202323
27808,202323_WesternBulldogs_We

Team Ratings - Combination of VAEP and Expected VAEP Values should be a correct order (Season by Season)

In [5]:
player_stats.head()

,Match_ID,Team,Player,Round_ID,AFL_API_Player_ID,Player_Type,playerId,Age,Height,Weight,Number,Kicking_Foot,State_Of_Origin,Draft_Year,Debut_Year,Recruited_From,Draft_Position,Draft_Type,Photo_URL,Date_Of_Birth,Percent_Played,Behinds,Bounces,Centre_Bounces_Attended,Centre_Clearances,Clangers,Defensive_Contest_Losses,Defensive_Contest_Loss_Percentage,Defensive_One_On_One_Contests,Contested_Marks,Contested_Possession_Rate,Contested_Possessions,Offensive_One_On_One_Contests,Offensive_Contest_Wins,Offensive_Contest_Win_Percentage,Defensive_Half_Pressure_Acts,Disposal_Efficiency,Disposals,AFL_Fantasy_Points,Effective_Disposals,Effective_Kicks,Inside_50_Ground_Ball_Gets,Frees_Against,Frees_For,Goal_Accuracy,Goal_Assists,Goals,Ground_Ball_Gets,Handballs,Hit_Outs,Hit_Outs_To_Advantage,Hit_Outs_To_Advantage_Rate,Hit_Out_Win_Percentage,Inside_50s,Intercept_Marks,Intercepts,Kick_Efficiency,Kick_Ins,Kick_Ins_Played_On,Kicks,Kick_To_Handball_Ratio,Marks,Marks_Inside_50,Marks_On_Lead,Metres_Gained,One_Percenters,Pressure_Acts,Player_Rating_Points,Rebound_50s,Ruck_Contests,Score_Involvements,Score_Launches,Shots_At_Goal,Spoils,Stoppage_Clearances,Tackles,Tackles_Inside_50,Clearances,Possessions,Turnovers,Uncontested_Possessions,AFLCA_Player_ID,Coaches_Votes,Position,Team_Status,Position_Sub_Group,Position_Group,Year,Brownlow_Votes,Season,xScore,xT_created,xT_denied,vaep_value,offensive_value,defensive_value,exp_vaep_value,exp_offensive_value,exp_defensive_value,xT_received,xT_prevented,vaep_value_received,exp_vaep_value_received,Player_Season,Score,xScore_Diff,Home_Team,Away_Team,Opponent,Round,Round_str,Round_ID_num
0,202101_BrisbaneLions_Sydney,Brisbane Lions,Brandon Starcevich,202101,Brandon_Starcevich,MEDIUM_DEFENDER,CD_I1002401,22,187,92,37,RIGHT,WA,2017.0,2018.0,Mt Lawley-Inglewood (WA)/Trinity Coll (WA)/Eas...,18.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1999-07-24,84.0,0.0,1.0,0.0,0.0,1.0,1.0,100.0,1.0,1.0,55.6,5.0,0.0,0.0,0.0,4.0,88.9,9.0,31.0,8.0,5.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,83.3,0.0,0.0,6.0,2.0,2.0,0.0,1.0,172.0,1.0,5.0,6.1,2.0,0.0,4.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,9.0,2.0,4.0,NaN,NaN,Half Back Flank Left,FINAL_TEAM,Half-Back,Back,2021.0,0.0,2021,0.000000,-0.235667,0.000000,0.711460,0.026457,0.685004,3.362637,0.585132,2.777505,-0.461340,0.190552,0.329877,2.328707,Brandon Starcevich_2021,0.0,0.000000,Brisbane Lions,Sydney,Sydney,1,01,20211
1,202101_BrisbaneLions_Sydney,Brisbane Lions,Callum Ah Chee,202101,Callum_Ah_Chee,MEDIUM_DEFENDER,CD_I298437,24,183,79,4,RIGHT,WA,2015.0,2016.0,Kelmscott (WA)/South Fremantle (WAFL)/Gold Coast,8.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1997-10-09,87.0,0.0,0.0,0.0,0.0,4.0,1.0,100.0,1.0,0.0,66.7,8.0,0.0,0.0,0.0,5.0,83.3,12.0,36.0,10.0,5.0,0.0,2.0,0.0,0.0,1.0,0.0,8.0,6.0,0.0,0.0,0.0,0.0,2.0,0.0,7.0,83.3,1.0,1.0,6.0,1.0,0.0,0.0,0.0,258.0,2.0,7.0,8.0,1.0,0.0,2.0,1.0,0.0,2.0,0.0,3.0,1.0,0.0,12.0,6.0,4.0,NaN,NaN,Interchange,FINAL_TEAM,Interchange,Interchange,2021.0,0.0,2021,0.000000,-0.075963,0.010892,0.291451,0.479986,-0.188535,1.318957,1.031427,0.287530,-0.487945,-0.039690,0.608221,3.269039,Callum Ah Chee_2021,0.0,0.000000,Brisbane Lions,Sydney,Sydney,1,01,20211
2,202101_BrisbaneLions_Sydney,Brisbane Lions,Charlie Cameron,202101,Charlie_Cameron,MEDIUM_FORWARD,CD_I990609,27,180,75,23,RIGHT,WA,2014.0,2014.0,Newman Saints (WA)/Swan Districts (WAFL)/Adelaide,NaN,rookieElevation,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1994-07-05,77.0,0.0,0.0,9.0,3.0,0.0,0.0,0.0,0.0,0.0,50.0,5.0,0.0,0.0,0.0,3.0,70.0,10.0,39.0,7.0,4.0,0.0,0.0,2.0,0.0,1.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,57.1,0.0,0.0,7.0,2.3,2.0,1.0,2.0,207.0,1.0,9.0,10.1,0.0,0.0,4.0,1.0,0.0,0.0,0.0,1.0,0.0,3.0,10.0,0.0,5.0,NaN,NaN,Forward Pocket Right,FINAL_TEAM,Forward-Pocket,Forward,2021.0,0.0,2021,0.000000,0.460322,-0.166355,0.696752,0.681301,0.015451,6.034727,5.882581,0.152147,0.197894,-0.035491,0.859897,8.186274,Charlie Cameron_2021,0.0,0.000000,Brisbane L

In [6]:
sorted(list(set(player_stats['Round_ID'])))[-10:]

['202314',
 '202315',
 '202316',
 '202317',
 '202318',
 '202319',
 '202320',
 '202321',
 '202322',
 '202323']

In [44]:
def create_team_season_summary(player_stats, rounds = None):
    
    player_stats_copy = player_stats.copy()
    
    if rounds is not None:
        all_rounds = sorted(list(set(player_stats_copy['Round_ID'])))
        rating_rounds = all_rounds[-rounds:]
        player_stats_copy = player_stats_copy[player_stats_copy['Round_ID'].isin(rating_rounds)]
    
    team_for = player_stats_copy.groupby(['Team', 'Season']).sum()[['Score', 'xScore','vaep_value', 'vaep_value_received', 'offensive_value', 'defensive_value', 'exp_vaep_value', 'exp_vaep_value_received', 'exp_offensive_value', 'exp_defensive_value']]
    team_for.columns = [x+"_F" for x in list(team_for)]
    team_against = player_stats_copy.groupby(['Opponent', 'Season']).sum()[['Score', 'xScore', 'vaep_value', 'vaep_value_received', 'offensive_value', 'defensive_value', 'exp_vaep_value', 'exp_vaep_value_received', 'exp_offensive_value', 'exp_defensive_value']]
    team_against.columns = [x+"_A" for x in list(team_against)]
    team_games = player_stats_copy.groupby(['Match_ID', 'Team', 'Season']).size().reset_index().groupby(['Team', 'Season']).count()[['Match_ID']]
    team_games.columns = ['Games']
    team_summary = pd.concat([team_games, team_for, team_against], axis=1)
    
    for col in [x.replace("_F", "") for x in list(team_for)]:
        team_summary[col+"_pergame_F"] = team_summary[col+"_F"] / team_summary["Games"]
        team_summary[col+"_pergame_A"] = team_summary[col+"_A"] / team_summary["Games"]

        team_summary[col+"_diff"] = team_summary[col+"_F"] - team_summary[col+"_A"]
        team_summary[col+"_pergame_diff"] = team_summary[col+"_pergame_F"] - team_summary[col+"_pergame_A"]

    return team_summary


In [45]:
def create_team_summary(player_stats, rounds = None):
    
    player_stats_copy = player_stats.copy()
    
    if rounds is not None:
        all_rounds = sorted(list(set(player_stats_copy['Round_ID'])))
        rating_rounds = all_rounds[-rounds:]
        player_stats_copy = player_stats_copy[player_stats_copy['Round_ID'].isin(rating_rounds)]
    
    team_for = player_stats_copy.groupby(['Team']).sum()[['Score', 'xScore','vaep_value', 'vaep_value_received', 'offensive_value', 'defensive_value', 'exp_vaep_value', 'exp_vaep_value_received', 'exp_offensive_value', 'exp_defensive_value']]
    team_for.columns = [x+"_F" for x in list(team_for)]
    team_against = player_stats_copy.groupby(['Opponent']).sum()[['Score', 'xScore', 'vaep_value', 'vaep_value_received', 'offensive_value', 'defensive_value', 'exp_vaep_value', 'exp_vaep_value_received', 'exp_offensive_value', 'exp_defensive_value']]
    team_against.columns = [x+"_A" for x in list(team_against)]
    team_games = player_stats_copy.groupby(['Match_ID', 'Team']).size().reset_index().groupby(['Team']).count()[['Match_ID']]
    team_games.columns = ['Games']
    team_summary = pd.concat([team_games, team_for, team_against], axis=1)
    
    for col in [x.replace("_F", "") for x in list(team_for)]:
        team_summary[col+"_pergame_F"] = team_summary[col+"_F"] / team_summary["Games"]
        team_summary[col+"_pergame_A"] = team_summary[col+"_A"] / team_summary["Games"]

        team_summary[col+"_diff"] = team_summary[col+"_F"] - team_summary[col+"_A"]
        team_summary[col+"_pergame_diff"] = team_summary[col+"_pergame_F"] - team_summary[col+"_pergame_A"]

    return team_summary

In [46]:
team_summary = create_team_season_summary(player_stats, rounds = None)
team_summary

Games  Score_F     xScore_F  vaep_value_F  \
                       Season                                              
Adelaide               2021       21   1474.0  1617.311226    348.047684   
                       2022       22   1672.0  1698.648109    378.075966   
                       2023       22   2031.0  2011.055823    436.994147   
Brisbane Lions         2021       24   2220.0  2235.584786    462.206382   
                       2022       25   2287.0  2235.635213    494.811894   
                       2023       21   1894.0  1895.003909    398.261358   
Carlton                2021       22   1701.0  1827.291812    380.457765   
                       2022       22   1808.0  1895.020471    394.554726   
                       2023       22   1784.0  1864.199045    386.938516   
Collingwood            2021       22   1516.0  1593.485306    339.534818   
                       2022       25   2001.0  2049.503781    444.044761   
                       2023       21   1912.0  1856.738253    390.921564   
Essendon               2021       23   1946.0  1940.093531    412.117911   
                       2022       22   1675.0  1727.511867    370.589703   
                       2023       22   1763.0  1792.031325    386.720312   
Fremantle              2021       22   1534.0  1713.151425    326.313035   
                       2022       24   1822.0  1929.771266    411.718442   
                       2023       21   1634.0  1705.687024    359.987703   
Geelong                2021       24   1894.0  1942.177685    409.387128   
                       2022       25   2406.0  2340.693751    472.416874   
                       2023       22   1955.0  1925.573813    388.541050   
Gold Coast             2021       22   1386.0  1542.701851    329.705621   
                       2022       22   1819.0  1930.030220    397.632774   
                       2023       22   1671.0  1781.092058    374.338930   
Greater Western Sydney 2021       24   1864.0  1938.951091    412.456883   
                       2022       22   1599.0  1609.333236    365.775899   
                       2023       21   1789.0  1832.310424    386.757594   
Hawthorn               2021       22   1579.0  1579.570844    363.791422   
                       2022       22   1740.0  1732.773358    390.516553   
                       2023       20   1453.0  1540.348694    323.218688   
Melbourne              2021       25   2180.0  2275.102274    450.244989   
                       2022       24   1990.0  2062.313749    424.312572   
                       2023       21   1860.0  1813.134917    397.006168   
North Melbourne        2021       22   1414.0  1518.841712    325.647459   
                       2022       22   1296.0  1448.195631    323.473597   
                       2023       21   1396.0  1441.353940    315.513566   
Port Adelaide          2021       24   1977.0  2006.226661    426.781382   
                       2022       22   1764.0  1789.647739    378.888675   
                       2023       21   1931.0  1995.033521    411.660096   
Richmond               2021       22   1701.0  1768.392739    356.807720   
                       2022       23   2209.0  2224.872230    453.403114   
                       2023       21   1662.0  1749.180816    358.370385   
St Kilda               2021       22   1606.0  1704.449970    351.753303   
                       2022       22   1650.0  1723.653053    358.586900   
                       2023       21   1596.0  1738.564291    352.319750   
Sydney                 2021       23   2013.0  1933.325213    433.103219   
                       2022       25   2232.0  2225.068690    475.291885   
                       2023       21   1824.0  1826.804274    393.767512   
West Coast             2021       22   1710.0  1652.462268    366.421181   
                       2022       22   1387.0  1315.676539    312.701158   
                       2023       22   1204.0  1369.361537    305.657384   
Wes

In [36]:
def zscale_stats(summary_data, stats):
    
    for col in stats:
        summary_data[col+'_z'] = (summary_data[col] - summary_data[col].mean()) / np.std(summary_data[col])
    summary_data = summary_data.fillna(0)
    
    return summary_data

In [38]:
def create_team_ratings(team_summary, rating_factors = []):
    
    team_ratings = team_summary[rating_factors]
    team_ratings = zscale_stats(team_ratings, rating_factors)
    team_ratings['team_rating'] = team_ratings[[x+"_z" for x in rating_factors]].mean(axis=1)
    
    return team_ratings.sort_values(by = 'team_rating', ascending = False)

In [47]:
team_summary = create_team_season_summary(player_stats, rounds = None)
team_ratings = create_team_ratings(team_summary, rating_factors = ['Score_pergame_diff', 'xScore_pergame_diff', 'vaep_value_pergame_diff', 'exp_vaep_value_pergame_diff'])
team_ratings

,,Score_pergame_diff,xScore_pergame_diff,vaep_value_pergame_diff,exp_vaep_value_pergame_diff,Score_pergame_diff_z,xScore_pergame_diff_z,vaep_value_pergame_diff_z,exp_vaep_value_pergame_diff_z,team_rating
,Season,,,,,,,,,
Geelong,2022,31.640000,29.009371,4.245756,27.884178,1.820618,1.812704,1.696240,1.785487,1.778762
Melbourne,2021,24.840000,27.547342,3.314383,26.167210,1.434869,1.722612,1.329761,1.677115,1.541089
Western Bulldogs,2021,19.538462,19.154052,2.988902,19.176889,1.134124,1.205411,1.201689,1.235894,1.194280
Brisbane Lions,2021,20.333333,19.192769,2.836990,18.592279,1.179215,1.207797,1.141915,1.198994,1.181980
Melbourne,2022,15.958333,17.086683,2.308653,16.204013,0.931031,1.078018,0.934023,1.048250,0.997830
Richmond,2022,16.521739,15.911164,2.439215,14.157343,0.962992,1.005582,0.985397,0.919067,0.968259
Melbourne,2023,16.761905,13.174047,2.744222,12.537216,0.976616,0.836919,1.105412,0.816806,0.933938
Brisbane Lions,2023,15.238095,15.840140,1.933281,13.935466,0.890174,1.001205,0.786320,0.905062,0.895690
Sydney,2022,15.760000,12.270852,2.437663,12.877271,0.919780,0.781263,0.984786,0.838270,0.881025


In [48]:
def get_cumulative_round_summary(player_stats, current_round_id, rating_factor, num_rounds = None):
    
    current_season = int(current_round_id[:4])
    if "F" in current_round_id:
        current_round_id_int = int(finals_round_map[current_round_id[4:]])
        current_round_id_str = finals_round_map[current_round_id[4:]]
    else:
        current_round_id_int = int(current_round_id[4:])
        current_round_id_str = current_round_id[4:]
    current_round_id_num = int(str(current_season) + str(current_round_id_str))

    past_seasons_player_stats = player_stats[(player_stats['Season'] < current_season)]
    current_round_player_stats = player_stats[(player_stats['Season'] == current_season) & (player_stats['Round'] <= current_round_id_int)]
    rating_player_stats = pd.concat([past_seasons_player_stats, current_round_player_stats], axis=0)

    round_summary_dict = {}
    for round_id in list(set(rating_player_stats['Round_ID'])):
        season = int(round_id[:4])
        if "F" in round_id:
            round_id_int = int(finals_round_map[round_id[4:]])
            round_id_str = finals_round_map[round_id[4:]]
        else:
            round_id_int = int(round_id[4:])
            round_id_str = round_id[4:]
        seasons_player_stats = rating_player_stats[(rating_player_stats['Season'] < season)]
        round_player_stats = rating_player_stats[(rating_player_stats['Season'] == season) & (rating_player_stats['Round'] <= round_id_int)]
        latest_round_player_stats = pd.concat([seasons_player_stats, round_player_stats], axis=0)
        
        latest_round_team_summary = create_team_summary(latest_round_player_stats, rounds = num_rounds)
        
        round_summary_dict[int(str(season) + str(round_id_str))] = latest_round_team_summary[rating_factor]
        
    round_summary_df = pd.DataFrame.from_dict(round_summary_dict)
    # round_summary_df.T.sort_index()
    round_summary_df = round_summary_df.reindex(sorted(round_summary_df.columns), axis=1).sort_values(by=int(current_round_id_num), ascending=False)
    return round_summary_df

In [49]:
xscore_f_summary = get_cumulative_round_summary(player_stats, current_round_id='202323', rating_factor = 'xScore_pergame_F', num_rounds = 10)
xscore_f_summary

,202101,202102,202103,202104,202105,202106,202107,202108,202109,202110,202111,202112,202113,202114,202115,202116,202117,202118,202119,202120,202121,202122,202123,202124,202125,202126,202127,202201,202202,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212,202213,202214,202215,202216,202217,202218,202219,202220,202221,202222,202223,202224,202225,202226,202227,202301,202302,202303,202304,202305,202306,202307,202308,202309,202310,202311,202312,202313,202314,202315,202316,202317,202318,202319,202320,202321,202322,202323
Carlton,84.197005,80.916275,94.170831,96.014360,93.691386,92.292777,94.283868,92.814433,89.829921,88.161757,87.109941,87.897614,84.255133,82.211379,83.213148,81.958491,77.824745,79.736297,81.566776,82.265521,80.437207,78.765945,78.648312,78.147995,76.239568,76.558390,77.813672,78.813835,77.337026,75.340948,78.758755,82.438127,79.202410,86.063949,88.956862,90.245759,91.121945,86.708798,88.168966,88.960490,88.780163,87.554606,92.523238,91.133704,85.118728,85.698579,82.582922,84.542328,84.024997,84.761493,85.723001,86.248734,83.870981,78.728247,79.476105,77.220654,80.020944,82.947058,79.640493,79.326902,87.645736,85.101119,82.682512,80.950374,81.640782,76.907157,76.404238,78.988193,81.150256,83.314718,79.006424,84.089353,91.579918,93.635301,93.178419,94.388845,95.620636
Sydney,92.468489,96.490469,102.182044,100.628085,95.712307,86.909251,85.298589,82.317101,83.265167,81.540361,80.717459,80.438926,73.983516,71.540994,70.882156,77.890592,79.667315,81.484403,82.895318,85.080151,82.337013,81.248963,88.371843,88.694392,90.758253,88.856797,88.461322,91.668544,88.478048,84.456936,86.939191,89.173243,90.319563,89.318182,86.615880,88.193400,86.663419,85.603699,85.840392,88.985678,88.226266,86.307723,83.309017,86.548525,87.633269,88.850810,93.799860,97.231192,97.384158,97.617142,98.233830,99.559127,99.106501,91.211632,94.132920,95.032392,91.792972,87.844194,88.204958,81.170510,84.447583,82.925773,80.597568,85.499216,81.659712,76.883842,72.774303,70.361320,83.511043,90.129111,86.172077,87.509136,89.017708,89.453630,89.816930,90.276809,93.270885
Brisbane Lions,69.728083,76.918707,77.877148,71.201923,78.109112,80.580829,83.019726,85.769847,88.494462,89.130849,93.661811,91.932190,93.280875,97.650889,96.650696,96.558666,94.348315,92.964967,95.600101,94.211471,94.321791,99.776944,101.780185,98.812394,97.518297,98.119091,100.663984,100.050570,93.155466,100.662080,94.289952,91.524391,88.661559,92.896551,93.951944,92.690686,92.831567,94.961683,95.449535,91.028838,93.914979,89.513752,90.050373,89.147433,90.052309,90.711345,91.845605,91.029563,91.531172,87.403967,87.993514,90.361435,86.051300,85.992340,80.539173,80.273236,75.574758,76.544266,75.082328,80.178521,81.655258,83.061920,88.403869,90.168016,93.879860,94.847528,96.276908,95.959435,94.860025,97.162897,98.730982,98.810089,94.648051,91.310045,90.350962,90.294019,92.532492
Adelaide,NaN,108.378634,99.162397,98.131887,90.098748,91.609992,88.482220,81.910835,82.124786,82.253137,82.445489,79.407239,78.303426,76.329267,77.366000,74.788185,70.733286,72.140082,72.283369,70.412942,67.509906,65.991860,70.661352,70.661352,70.077779,69.452515,74.969722,79.566462,76.310992,79.279103,85.479533,87.913017,82.335251,79.562066,77.389196,76.337936,77.601022,75.607084,77.661299,76.928692,75.555070,78.161808,74.910962,74.212102,75.682634,76.165626,76.824121,78.368501,79.536171,77.393178,76.782496,73.580899,75.776848,78.967455,82.702821,85.221361,86.509384,88.877042,90.790629,94.926008,92.019632,89.023677,90.826187,87.141269,85.749490,85.438132,89.338281,89.165855,86.188439,91.081282,90.936579,90.286817,88.793229,95.075961,97.640794,99.412688,92.204167
Port Adelaide,115.529080,107.691528,93.419407,93.735986,92.188816,91.187074,86.660801,88.115299,85.973642,83.180476,81.235799,79.167113,80.916673,79.261435,78.712765,79.061282,78.188233,76.289045,79.846014,82.698142,78.773756,84.987123,84.195671,85.293310,85.763443,83.061409,86.988369,85.144635,82.062787,83.305801,81.259383,73.774160,76.9

In [50]:
xscore_a_summary = get_cumulative_round_summary(player_stats, current_round_id='202323', rating_factor = 'xScore_pergame_A', num_rounds = 10).sort_values(by=202323, ascending=True)
xscore_a_summary[[x for x in list(xscore_a_summary) if '2023' in str(x)]]

,202301,202302,202303,202304,202305,202306,202307,202308,202309,202310,202311,202312,202313,202314,202315,202316,202317,202318,202319,202320,202321,202322,202323
Carlton,79.771067,81.703919,77.671763,75.450737,80.259048,82.948715,77.053212,78.559869,77.278996,78.431750,77.522278,78.726952,79.737046,77.727323,74.373480,71.119878,73.315729,73.278618,72.665467,73.391364,71.101399,68.265446,68.850166
Greater Western Sydney,92.341902,90.304829,85.360394,94.846260,101.179360,99.662573,100.131437,96.028495,97.310171,97.469410,95.045491,96.613381,94.672178,88.004902,88.004902,85.074210,82.207477,81.184509,74.927045,73.038702,72.645079,72.494159,69.909033
St Kilda,81.540533,80.826323,80.909635,76.510405,74.164532,70.235261,72.380789,71.298053,75.541579,74.668005,78.985497,82.362419,80.156755,81.732895,82.214212,79.652598,77.094236,78.562087,73.765786,75.075582,69.924863,70.666635,70.887670
Brisbane Lions,91.608441,89.181519,85.096605,85.832793,87.500701,86.492561,84.144595,81.510593,77.507384,75.939288,70.351674,70.883900,72.702694,70.181538,70.036686,67.135750,63.581061,64.614032,64.448330,68.377362,67.211581,70.512372,71.526782
Melbourne,73.054997,72.275213,76.524568,75.143025,78.522636,81.313043,77.670349,78.256225,74.817222,77.436716,77.809127,73.685939,71.124953,71.246452,68.084884,63.661411,65.976802,66.785496,71.729804,69.938187,71.159706,72.370132,73.582596
Western Bulldogs,78.941552,77.230408,72.952820,73.014992,77.647570,78.560288,76.501663,76.013110,74.604282,72.541554,70.183265,70.563301,74.255206,73.950197,72.925941,74.377558,76.122389,77.254787,75.980414,79.790788,79.437895,79.018421,77.253497
Sydney,73.041837,72.664146,79.609620,77.583914,80.665672,84.254658,85.798049,85.200973,87.181869,84.299341,84.979726,86.992746,83.843353,88.174002,82.445222,78.247574,80.400577,80.018905,76.337439,77.131474,78.882999,78.952395,78.421891
Adelaide,94.329530,90.593858,92.603015,92.092489,87.371520,82.355433,79.209497,80.102288,79.665021,82.195224,79.932683,80.988155,77.915530,78.653281,81.678417,79.488047,84.268438,82.898618,85.548788,81.795707,79.125506,77.999887,81.672035
Fremantle,74.658469,73.948526,73.318907,74.219090,79.283186,84.389571,87.730875,84.259263,82.577724,82.532562,83.951831,86.531893,85.366078,86.801093,84.795079,80.624691,79.921443,84.651256,86.159828,85.313907,84.488645,80.914380,81.819682
Collingwood,81.974677,80.351317,78.347849,79.970826,79.347419,77.721359,76.418318,77.588789,75.311826,74.206194,72.873144,71.697130,75.461692,71.101204,72.213867,70.829923,71.208002,71.954239,73.346031,75.401414,78.112270,81.813738,82.623498


In [51]:
get_cumulative_round_summary(player_stats, current_round_id='202323', rating_factor = 'exp_vaep_value_pergame_diff', num_rounds=10)

,202101,202102,202103,202104,202105,202106,202107,202108,202109,202110,202111,202112,202113,202114,202115,202116,202117,202118,202119,202120,202121,202122,202123,202124,202125,202126,202127,202201,202202,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212,202213,202214,202215,202216,202217,202218,202219,202220,202221,202222,202223,202224,202225,202226,202227,202301,202302,202303,202304,202305,202306,202307,202308,202309,202310,202311,202312,202313,202314,202315,202316,202317,202318,202319,202320,202321,202322,202323
Carlton,-37.800390,-25.606683,1.174708,12.632183,10.620424,7.235182,8.391625,4.573764,1.767470,2.256994,5.299252,7.064046,1.766997,-3.807684,-1.937663,-1.351894,-3.187549,3.616098,5.235327,3.837945,3.329216,-4.878879,-8.204208,-8.763840,-12.284767,-13.408433,-15.444736,-17.684425,-19.524551,-17.332168,-19.685963,-6.331162,-5.366540,4.820026,9.864239,13.113335,14.009120,8.186960,11.002242,11.426202,13.664920,14.174614,19.699846,19.229854,10.499835,10.226994,5.297664,6.430785,5.694039,6.611601,8.005188,7.121066,5.655126,-3.687269,-0.890323,-3.537382,3.814615,8.150968,0.000515,-3.632727,9.897847,6.360531,5.166388,2.523322,4.097216,-1.855179,-3.361409,1.580050,7.293322,12.772410,6.498143,11.486126,19.545649,20.904904,22.849123,26.510234,26.070458
Greater Western Sydney,15.619497,-11.459450,-18.874238,-13.754749,-7.685667,-13.814229,-5.975894,-3.449112,-2.243230,1.056543,-5.179600,-1.473067,3.526398,3.700155,-0.411178,4.350648,0.609872,-0.439227,-0.041337,-5.007145,-0.093306,2.668116,5.352196,1.792829,0.567551,0.808457,-0.013877,-1.040192,-5.507450,-0.854559,-5.311262,-13.366539,-20.048212,-11.509797,-12.097589,-15.098534,-9.155257,-12.609620,-11.260331,-10.623473,-10.586412,-10.445578,-7.257440,-13.708732,-13.914797,-13.641956,-26.412502,-18.717915,-18.108479,-24.030260,-22.496127,-20.851418,-25.740484,-26.333822,-18.741107,-12.815164,-4.684918,-15.051013,-15.536504,-14.055597,-14.349289,-12.298636,-14.248535,-16.335003,-15.394478,-16.926516,-9.677691,-1.070062,-1.070062,-2.680573,1.667071,3.045046,12.215504,16.079020,15.231939,13.714415,18.601197
Brisbane Lions,-20.508690,-10.518305,-3.121426,-10.698326,2.246935,3.487617,8.086144,10.429470,16.130000,17.657124,24.382186,21.484922,22.575210,31.038360,29.106689,29.870971,24.221925,21.078263,20.305905,16.939385,18.308475,27.143805,26.656098,19.942874,17.301982,17.383363,21.451797,24.297899,17.454048,29.146206,18.766349,14.734917,13.614332,17.944519,23.411644,21.546420,18.249025,19.913932,19.407545,12.974088,17.079556,9.202503,10.026538,6.701860,5.062971,4.662053,6.280852,4.550008,6.961133,1.000190,1.945978,5.449048,-0.277131,1.878023,-12.216485,-10.493026,-11.294172,-11.966036,-15.545350,-9.777920,-5.623280,-1.061093,9.143160,12.702987,23.000023,23.559089,22.770852,25.671630,24.409886,28.977457,33.321530,32.394562,28.220534,20.903375,21.246693,17.196577,18.364867
Sydney,20.508690,8.659175,16.315894,16.778780,10.104892,-1.220840,-6.507416,-4.741023,-1.061325,-3.469883,-4.782972,-3.428865,-9.584680,-12.668249,-12.493254,0.336678,4.965798,3.585777,3.963817,6.840321,2.650098,3.685757,12.862332,14.818714,18.133650,13.197870,14.592535,15.765467,11.420257,5.327877,10.335987,15.881886,15.567731,9.471340,5.898334,10.434274,7.183728,7.615864,9.008005,14.665009,12.185303,10.198904,5.980077,9.945516,13.949081,13.312811,23.609686,25.855830,27.625355,27.121815,29.602408,28.378747,28.037510,18.458084,21.482653,22.048870,12.585859,10.378527,7.774682,-2.617852,-1.542027,-2.710527,-6.103213,1.258320,-3.548867,-9.753891,-11.023032,-16.976064,0.439169,10.952038,4.747986,6.746685,11.203738,10.995740,10.101066,10.831652,13.886377
Adelaide,NaN,3.190341,10.447667,15.579025,8.319056,5.573065,-2.198131,-9.861202,-10.057456,-10.088949,-9.674521,-8.133301,-7.807289,-11.546070,-12.206373,-13.446224,-16.301368,-14.590726,-12.318108,-15.265868,-15.274041,-20.834224,-17.251296,-17.251296,-16.982261,-17.041219,-8.756354,-2.739937,-9.583755,-4.866035,-1.932491,1.462702,-1.718435,-6.59882

In [52]:
def get_cumulative_round_team_ratings(player_stats, current_round_id, rating_factors, num_rounds = None):
    
    current_season = int(current_round_id[:4])
    if "F" in current_round_id:
        current_round_id_int = int(finals_round_map[current_round_id[4:]])
        current_round_id_str = finals_round_map[current_round_id[4:]]
    else:
        current_round_id_int = int(current_round_id[4:])
        current_round_id_str = current_round_id[4:]
    current_round_id_num = int(str(current_season) + str(current_round_id_str))

    past_seasons_player_stats = player_stats[(player_stats['Season'] < current_season)]
    current_round_player_stats = player_stats[(player_stats['Season'] == current_season) & (player_stats['Round'] <= current_round_id_int)]
    rating_player_stats = pd.concat([past_seasons_player_stats, current_round_player_stats], axis=0)

    round_summary_dict = {}
    for round_id in list(set(rating_player_stats['Round_ID'])):
        season = int(round_id[:4])
        if "F" in round_id:
            round_id_int = int(finals_round_map[round_id[4:]])
            round_id_str = finals_round_map[round_id[4:]]
        else:
            round_id_int = int(round_id[4:])
            round_id_str = round_id[4:]
        seasons_player_stats = rating_player_stats[(rating_player_stats['Season'] < season)]
        round_player_stats = rating_player_stats[(rating_player_stats['Season'] == season) & (rating_player_stats['Round'] <= round_id_int)]
        latest_round_player_stats = pd.concat([seasons_player_stats, round_player_stats], axis=0)
        
        latest_round_team_summary = create_team_summary(latest_round_player_stats, rounds = num_rounds)
        latest_round_team_ratings = create_team_ratings(latest_round_team_summary, rating_factors = rating_factors)
        round_summary_dict[int(str(season) + str(round_id_str))] = latest_round_team_ratings['team_rating']
        
    round_summary_df = pd.DataFrame.from_dict(round_summary_dict)
    round_summary_df = round_summary_df.reindex(sorted(round_summary_df.columns), axis=1).sort_values(by=int(current_round_id_num), ascending=False)
    return round_summary_df

In [53]:
get_cumulative_round_team_ratings(player_stats, current_round_id='202323', rating_factors = ['xScore_pergame_diff', 'exp_vaep_value_pergame_diff'], num_rounds=10)

,202101,202102,202103,202104,202105,202106,202107,202108,202109,202110,202111,202112,202113,202114,202115,202116,202117,202118,202119,202120,202121,202122,202123,202124,202125,202126,202127,202201,202202,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212,202213,202214,202215,202216,202217,202218,202219,202220,202221,202222,202223,202224,202225,202226,202227,202301,202302,202303,202304,202305,202306,202307,202308,202309,202310,202311,202312,202313,202314,202315,202316,202317,202318,202319,202320,202321,202322,202323
Carlton,-1.445822,-1.106823,0.074792,0.591169,0.450016,0.321638,0.434446,0.252572,0.109913,0.134066,0.301612,0.421405,0.098387,-0.231245,-0.123018,-0.102878,-0.249429,0.282339,0.401378,0.264343,0.235474,-0.361300,-0.522152,-0.573904,-0.757950,-0.756276,-0.738551,-0.918014,-1.106308,-0.958310,-1.203523,-0.265418,-0.174276,0.295620,0.510926,0.617713,0.674151,0.385039,0.461536,0.508568,0.586995,0.652198,0.879986,0.943298,0.581554,0.641848,0.348501,0.440647,0.386360,0.398389,0.514044,0.456274,0.409234,-0.078856,0.053551,-0.143784,0.308401,0.550081,0.073462,-0.197415,0.725648,0.480819,0.341681,0.168187,0.257740,-0.099887,-0.175062,0.077195,0.310150,0.527733,0.259324,0.478270,0.837481,0.996841,1.095217,1.281588,1.399487
Brisbane Lions,-0.817521,-0.457068,-0.128001,-0.495662,0.093674,0.142745,0.410081,0.562493,0.956068,0.997661,1.394331,1.251060,1.491542,1.938657,2.136687,2.269480,1.819859,1.723794,1.654194,1.332580,1.418508,1.907442,1.721614,1.423965,1.214454,1.196917,1.301220,1.527337,1.200128,1.908395,1.325765,0.881162,0.719287,0.887361,1.098940,0.988344,0.870181,0.902806,0.823487,0.579728,0.743181,0.440339,0.454398,0.335753,0.284368,0.251438,0.323169,0.240460,0.377937,0.029172,0.112951,0.323856,0.055046,0.229008,-0.586234,-0.489677,-0.624541,-0.560328,-0.794104,-0.521083,-0.209536,0.061040,0.626532,0.871549,1.436201,1.300580,1.162742,1.266169,1.084590,1.246392,1.454695,1.427094,1.271077,1.061053,1.081621,0.902785,1.043636
Greater Western Sydney,0.525524,-0.553374,-0.777794,-0.607518,-0.349766,-0.632576,-0.294303,-0.183376,-0.126556,0.055042,-0.293599,-0.067221,0.222000,0.195553,-0.042969,0.305797,-0.019693,-0.133229,-0.098480,-0.469438,-0.090310,0.102110,0.268156,0.107724,0.047924,0.083040,0.074812,0.041614,-0.262013,0.023642,-0.301250,-0.718428,-0.938629,-0.531646,-0.559516,-0.682904,-0.432893,-0.565517,-0.474835,-0.464847,-0.427412,-0.442669,-0.278730,-0.612002,-0.682098,-0.750027,-1.505341,-1.124610,-1.074196,-1.263473,-1.196313,-1.056608,-1.405904,-1.456978,-1.016800,-0.662304,-0.170789,-0.777185,-0.831758,-0.899101,-0.848862,-0.750325,-0.832980,-1.035094,-0.947231,-0.936308,-0.527999,-0.086952,-0.084981,-0.131964,0.047599,0.102973,0.527705,0.781715,0.755154,0.689226,1.024621
Sydney,0.817521,0.346083,0.642456,0.761753,0.457546,-0.048052,-0.306452,-0.254657,-0.051181,-0.188137,-0.256601,-0.156110,-0.628694,-0.776004,-0.901542,0.044335,0.387458,0.375911,0.400178,0.608685,0.276353,0.315861,0.902261,1.111132,1.304828,0.974749,0.951785,1.022431,0.814233,0.422538,0.720394,0.917307,0.742515,0.419120,0.235389,0.425277,0.274452,0.292749,0.327501,0.578051,0.486074,0.440219,0.276685,0.498480,0.759748,0.775564,1.393021,1.611196,1.695937,1.484767,1.669969,1.542856,1.626463,1.167199,1.320337,1.448804,0.951158,0.700432,0.580136,-0.140507,-0.038507,-0.123127,-0.359058,0.085114,-0.206426,-0.542604,-0.563473,-0.843195,0.032969,0.482157,0.223248,0.304989,0.519419,0.564330,0.512601,0.545203,0.764114
Adelaide,NaN,0.235606,0.395254,0.695014,0.347681,0.237502,-0.146139,-0.566426,-0.614978,-0.596038,-0.578433,-0.500536,-0.574304,-0.771213,-0.945443,-1.070305,-1.245329,-1.235990,-1.065948,-1.237588,-1.191872,-1.474003,-1.088144,-1.132898,-1.061154,-0.987582,-0.381480,-0.045114,-0.506108,-0.207575,-0.082562,0.130469,-0.054925,-0.281696,-0.511249,-0.504882,-0.363314,-0.408774,-0.083991,-0.054971,-0.240617,-0.079511,-0.425836,-0.425654,-0.189192,-0.390914,-0.406908,-0.308996,-0.529231,-0.729379,-0.617936,-0.828358,-0.676514